In [3]:
import spacy
from spacy.tokens import DocBin #for the file format in spacy
from tqdm import tqdm #presenting progress with bar

nlp = spacy.load("en_core_web_sm") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json
f = open('./annotations.json')
TRAIN_DATA = json.load(f)

In [5]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 324/324 [00:00<00:00, 4262.16it/s]

Skipping entity


In [6]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.90    0.00    0.00    0.00    0.00
  2     200         72.01   2063.54   82.51   83.72   81.34    0.83
  5     400        300.88    657.42   94.62   94.72   94.52    0.95
 10     600        172.77    423.30   95.98   97.49   94.52    0.96
 15     800        582.48    456.99   95.82   96.31   95.33    0.96
 21    1000        220.48    368.10   96.32   97.11   95.54    0.96
 28    1200        146.16    345.65   96.85   96.95   96.75    0.97
 38    1400        186.02    350.07   97.05   97.35   9

[2022-06-20 20:55:51,816] [INFO] Set up nlp object from config
[2022-06-20 20:55:51,822] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-06-20 20:55:51,824] [INFO] Created vocabulary
[2022-06-20 20:55:51,824] [INFO] Finished initializing nlp object
[2022-06-20 20:55:52,030] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [9]:
nlp_ner = spacy.load("./model-best")

In [12]:
doc = nlp_ner('''Dhaka University is a bad institution''')
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

C:\Users\Johayra Prithula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\spacy\displacy\__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
